In [1]:
! pip install pebble
! pip install BeautifulSoup4
! pip install xlrd

     |████████████████████████████████| 102kB 15.0MB/s ta 0:00:01
     |████████████████████████████████| 112kB 16.7MB/s eta 0:00:01


In [2]:
import pandas as pd, requests, re, numpy as np, datetime, time
from bs4 import BeautifulSoup
from multiprocessing import Pool, Lock
from pprint import pprint
from pebble import ProcessPool
from concurrent.futures import TimeoutError
import pytz

In [3]:
# Общие функции 

def google(company_name, number_of_pages=1):
    """
    Поиск компании в Google по определенному количеству страниц(number_of_pages)
    Возвращает список всех сайтов по компании
    """
    s = requests.Session()
    company_name = '+'.join(company_name.split())
    url = 'https://www.google.com/search?q=' + company_name + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get,timeout = 10) 
    soup = BeautifulSoup(r.text, "html.parser")
    output = []
    for searchWrapper in soup.find_all('div', {'class':'r'}): #this line may change in future based on google's web page structure
        url = searchWrapper.find('a')["href"] 
        text = searchWrapper.find('a').text.strip()
        result = {'text': text, 'url': url}
        output.append(result) 
    if number_of_pages != 1:
        for page in range(2,number_of_pages+1):
            for searchWrapper in soup.find_all('div', {'class':'r'}): #this line may change in future based on google's web page structure
                url = searchWrapper.find('a')["href"] 
                text = searchWrapper.find('a').text.strip()
                result = {'text': text, 'url': url}
                output.append(result)
            url = soup.find('a', {'class':'fl','aria-label' : 'Page '+str(page)})
            #print(url['href'])
            r = s.get('https://www.google.com'+url['href'], headers=headers_Get,timeout = 10)
            soup = BeautifulSoup(r.text, "html.parser")        

    return output

def yandex(company_name, number_of_pages=1):
    """
    Поиск компании в Yandex по определенному количеству страниц(number_of_pages)
    Возвращает список всех сайтов по компании
    """
    s = requests.Session()
    company_name = '+'.join(company_name.split())
    #url = 'https://www.yandex.ru/search?text=' + company_name
    url = 'https://www.yandex.ru/search/?text=171%20%D0%9C%D0%95%D0%A0%D0%98%D0%94%D0%98%D0%90%D0%9D&lr=213&redircnt=1549970571.1'
    r = s.get(url, headers=headers_Get)
    r.encoding = 'utf-8'
    print(url)
    soup = BeautifulSoup(r.text, "html.parser",from_encoding = 'utf-8')
    print(r.text)
    output = []
    for page in range(2,number_of_pages+1):
        for searchWrapper in soup.find_all('div', {'class':'organic.typo.typo_text_m.typo_line_s'}): #this line may change in future based on google's web page structure
            print(soup.find_all('div', {'class':'organic.typo.typo_text_m.typo_line_s'}))
            url = searchWrapper.find('a')["href"] 
            text = searchWrapper.find('a').text.strip()
            result = {'text': text, 'url': url}
            output.append(result)
        print(output)
        url = soup.find('div', {'class':'pager.i-bem.pager_js_inited'}).a[page]['href']
        r = s.get('https://www.yandex.ru'+url['href'], headers=headers_Get)
        soup = BeautifulSoup(r.text, "html.parser")        

    return output

def load_data(file_name, number_of_rows = 0, start_row = 0,all_rows = True):
    """
    Выгрузка данных их файла excel начиная с опреденной строчки(start_row)
    по определенному количеству строк(number_of_rows)
    Возвращает список всех сайтов по компании
    """    
    
    data = pd.read_excel(file_name,header = 0)
    if all_rows == False:
        if start_row != 0 and number_of_rows != 0:
            data = data.iloc[start_row:start_row+number_of_rows]
        elif start_row != 0:
            data = data.iloc[start_row:data.shape[0]] 
        else:
            data = data.iloc[0:number_of_rows] 
    #data = data[data['Название компании'] == 'ТЕХНОЛОГИЯ ЦЕНТР']
    data["Найдено по"] = 'Информация не найдена'
    data['Ссылка'] = ''
    data['Время поиска'] = ''
    data['Комментарий'] = ''
    return data


def remove_avoided_cites(search_links, avoided_cites):
    """
    Удаляет из списка сайтов (search_links) заведомо ошибочные сайты(avoided_cites)
    Возвращает список скорректированных сайтов
    """   
    
    corrected_cites = []
    for search_link in search_links:
        failed = False
        for avoided_cite in avoided_cites:
            if search_link['url'].find(avoided_cite) != -1:
                # Доработка по добавлению сайта компании с hh.ru(Юля, письмо Fri 4/12/2019 3:41 PM)
                if search_link['url'].find('hh.ru') != -1:
                    try:
                        r = requests.get(search_link['url'],headers=headers_Get)
                        soup = BeautifulSoup(r.text,"html.parser")
                        company_site = soup.find('a', class_ = 'company-url')['href']
                        corrected_cites.append({'text':'hh.ru','url':company_site})
                        break
                    except:
                        failed = True
                else:
                    failed = True
        if failed != True:
            corrected_cites.append(search_link)       
    return corrected_cites


def change_adress(Adress):
    """
    Заменяет полные названия частей улиц на их сокращения(чтобы вдальнейшем удалить их regex)
    """
    try:
        Adress = Adress.lower().replace('проспект','пр-кт.')
        Adress = Adress.lower().replace('улица','ул.')
        Adress = Adress.lower().replace('дом','д.')
        Adress = Adress.lower().replace('проезд','пр-д.')
        Adress = re.sub(r'(^ул\s)|(\sул\s)|($\sул)','ул.',Adress)
        Adress = re.sub(r'(^наб\s)|(\sнаб\s)|($\sнаб)','наб.',Adress)
        Adress = re.sub(r'(^б\s)|(\sб\s)|($\sб)','б.',Adress)
    except:
        return Adress
    return Adress
    

def kick_all_external_links(domain,soup,cite_links):
    """
    Добавляет только внутренние ссылки в список сайтов домена
    """
    for link in soup.find_all('a'):
        try:
            a= link.get('href')
            link_block = False
            for avoided_extension in avoided_extensions:
                if link.get('href').find(avoided_extension) != -1:
                    link_block = True  
            if link_block == True:
                continue
        #Проверка на домен
            if link.get('href').find(domain) != -1 and link_block == False:
                cite_links.add(link.get('href'))

            elif (link.get('href').startswith("/") or link.get('href').startswith("#")) and re.match(r'(\..{1,4})$',link.get('href')) not in avoided_extensions and link_block == False:
                if link.get('href').startswith("#"):
                    link="http://"+domain+'/'+link.get('href')              
                else:
                    link="http://"+domain+link.get('href')
                cite_links.add(link)
        except:
            continue    
    cite_links = list(cite_links)
    corrected_cites = []
    for search_link in cite_links:
        failed = False
        for avoided_cite in avoided_cites:
            if search_link.find(avoided_cite) != -1:
                failed = True
                break
        if failed != True:
            corrected_cites.append(search_link) 
    return set(corrected_cites)

def isNaN(num):
    """
    Проверка на пустую ячейку
    """
    return num != num

def INN_check(INN):
    """
    Проверяет наличие корректного ИНН
    """
    if len(str(INN)) != 10:
        if isNaN(str(INN)):
            return "Нет ИНН"
        else:
             return "Длина ИНН не равна 10"
    return ''

def telefon_check(pattern, telefon):
    """
    Проверяет соотвествие телефона формату
    """
    if isNaN(telefon):
        return 'Номер телефона отсутствует'
    
    for group in re.finditer(pattern, telefon):
        if isNaN(group):
            return 'Телефон не соотвествует формату'
    if re.match(pattern,telefon) == None:
        return 'Телефон не соотвествует формату'
    return ''

def adress_check(city,adress_group_1, adress_group_2):
    if isNaN(city) or isNaN(adress_group_1) or isNaN(adress_group_2):
        return 'Адрес не соответствует формату'
    return ''
        

def INN_search(INN_pattern, soup):
    """
    Ищет ИНН в очищенном от всего кроме цифр и пробелов тексте
    """ 
    numbers_for_inn = re.sub("[^0-9 ]", "", soup)
    INN_check = re.search(INN_pattern,numbers_for_inn)
    return INN_check

def telefon_search(Telefon_patern,soup):
    """
    Заменяем весь текст кроме цифр для поиска номера
    """
    numbers_for_telefon_number = re.sub("[^0-9]", "", soup) 
    return numbers_for_telefon_number

def adress_search(City_pattern,Adress_group_1,Adress_group_2,soup):
    """
    Ищет адрес в тексте
    """
    City_check = re.search(City_pattern, soup, re.IGNORECASE)
    #print(f'Город поиска: {City_pattern} нашел {City_check}')
    Adress_check_1 = re.search(Adress_group_1, soup, re.IGNORECASE)
    #print(f'Первая группа поиска: {Adress_group_1}нашел {Adress_check_1}')
    #Adress_group_2 =  '22/25'
    Adress_check_2 = re.search(Adress_group_2, soup, re.IGNORECASE)
    #print(f'Вторая группа поиска: {Adress_group_2}нашел {Adress_check_2}')
    #print(soup)
    return (City_check,Adress_check_1,Adress_check_2)

def get_encoding(content):
    """
    Ищет кодировку сайта: если не находит, то берет кодировку utf-8
    """ 
    encoding_row = re.search(r'<meta charset="([^\t\r\n]*)">',content)
    try:
        encoding = encoding_row[1] 
    except:
        encoding = 'utf-8'
def load_page(url):
    try:
        with requests.get(url, headers=headers_Get,timeout = 10, stream=True) as r:
            maxsize = 1048576
            content = ''
            try:
                if int(r.headers['content-length']) < maxsize:  
                    r.encoding = get_encoding(r.text)
                    content = r.text
                else:
                    r.close()
                    return False
            except KeyError:
                r.encoding = get_encoding(r.text)
                content = r.text
            soup = BeautifulSoup(content, "html.parser")
    except:
        return False
    return soup

In [4]:
# Удаление агрегаторов

def kick_agregator(company,soup):
    agregator_words = {'ОКПО':50,
                       "ОГРН":25,
                        'ОКАТО':25,
                        'ОКТМО':50,
                        'ОКОГУ':50,
                        'ОКФС':50,
                        'ОКОПФ':50,
                        'ОКПО':50,
                        'Организационная форма':50,
                        'займы':50,
                        'ID компании' :100,
                        'Наименование':34,
                        'Тариф':20,
                        'Сфера деятельности':50,
                        'Обьявление':34,
                        'ОКВЭД':34,
                        'Справочн':34,
                        'Собственники':50,
                        'Уставной капитал':50,
                        'Численность персонала':100,
                       'Количество сотрудников' :100,
                        'Суд':34,
                        'Судья':50,
                        'Судебный орган':50,
                        'Справочник организаций':100,
                        'Кредитная история':50,
                        'ОКПД':50,
                        'Новый ОКВЭД-2':50,
                        'Вид деятельности':34,
                        'заработная':50,
                        'Похожие':20,
                        'Виды деятельности': 50,
                        'ОКП': 34,
                        'КПП':50,
                        'Идентификатор компании':100,
                        'описание организации' :50,
                        'реклама':20,
                        'рейтинг компании':50,
                        'сфера деятельности':50,
                        'Персоны':50,
                        'профиль компании' :50,
                        'сотрудники компании':50,
                        'вакансии': 20,
                        'резюме':50,
                        'добавить компанию':50,
                       'Недавно добавленные компании':100
                      }
    sum_rejection = 0
    for word in agregator_words.keys():
        if re.search(word,soup, re.IGNORECASE) != None:
            sum_rejection += agregator_words[word]
            if company.debug_check == 2:
                print(f"На сайте {company.search_link_url} было найдено слово-агрегатор /{word}/ :\nsum_rejection = {sum_rejection}")
        if sum_rejection >= 100:
            if company.debug_check == 2:
                print(f'Был выбит сайт {company.search_link_url} как агрегатор')
            company.INN_result = None
            company.Telefon_groups_result = None 
            company.Adress_result = (None,None,None)
            company.Comments += 'Был Выбит агрегатор'
            break
        #else:
            #print(company.INN_result)
            #print(company.Telefon_groups_result)
            #print(company.Adress_result)
            #print(company.Adress_group_2)

In [5]:
# Основной класс

class Company_data():
    """
    Основной класс, сохраняет в себе все необходимые данные по компании и 
    проводит сверку сайтов с информацией о компании
    """

    def __init__(self,row):
        self.Company_name = row[1]['Название компании'] +" Компания" # + "Компания" для облегчения поиска компаний со сложными названиями
        self.INN = row[1]['Номер налогоплательщика (ИНН)'] 
        self.INN = re.sub('[^0-9]',"",str(self.INN))
        self.City = row[1]['Город'] 
        self.Adress = row[1]['Адрес']
        self.Company_index = row[0]    
        self.Telefon_number = row[1]['Номер телефона'] 
        self.Telefon_number = re.sub('[^0-9]',"",str(self.Telefon_number))
        self.INN_patern = re.compile(str(self.INN))
        self.INN_check = INN_check(self.INN)
        self.Telefon_patern = re.compile(r'(7|8)?([0-9]{3})([0-9]{3})([0-9]{2})([0-9]{2})')
        self.Telefon_groups_correct = re.match(self.Telefon_patern,str(self.Telefon_number))  
        self.Telefon_check = telefon_check(self.Telefon_patern, self.Telefon_number)
        if self.Telefon_check == '':
            self.correct_telefon = (self.Telefon_groups_correct[2]+self.Telefon_groups_correct[3]+ 
                                    self.Telefon_groups_correct[4]+self.Telefon_groups_correct[5])
        else:
            self.correct_telefon = self.Telefon_check
        self.Adress_patern = re.compile(r'([()абвгдежзийклмнопёрстуфхцчшщьъыэюя.0-9/\\\s-]*),\s*([()абвгдежзикёлмнопрстуфхцчшщьъыэюя.0-9/\\\s-]*),?\s?([()абвгдежзиклмнопрстуфхцчшщёьъыэюя.0-9/\\\s-]*)', re.IGNORECASE)  
        self.Adress_groups_check = ''
        try:
            self.Adress = re.sub("[0-9]-[абвгдежзикёлмнопрстуфхцчшщьъыэюя]","",self.Adress, flags=re.I) # Зачем?
            self.Adress_groups_correct = re.match(self.Adress_patern,change_adress(self.Adress))
            self.Adress_group_1 = re.sub(r'(\s?[абвгдежзийклмнопёрстуфхцчшщьъыэюя0-9/\\-]*\.\s?)', '', self.Adress_groups_correct[1])
            #self.Adress_group_1 = re.sub(r'([^абвгдежзийклмнопёрстуфхцчшщьъыэюя][абвгдежзийклмнопёрстуфхцчшщьъыэюя][^абвгдежзийклмнопёрстуфхцчшщьъыэюя])|(\s[абвгдежзийклмнопёрстуфхцчшщьъыэюя])', '', self.Adress_group_1).strip()
            self.Adress_group_2 = re.sub(r'(\s?[абвгдежзийклмнопрстуфхцёчшщьъыэюя0-9/\\-]*\.\s?)', '', self.Adress_groups_correct[2]).strip() 
            self.Adress_groups_check = adress_check(self.City,self.Adress_group_1, self.Adress_group_2)
        except: 
            self.Adress_groups_check = 'Не может перевести адрес в regex'
        self.search_links = remove_avoided_cites(google(self.Company_name, google_number_of_pages), avoided_cites)
        self.additional_information = row[1]['Сайт компании']
        self.append_additinal_information()
        self.Answer_INN = ''
        self.Answer_telefon = ''
        self.Answer_adress = ''
        self.stop_search = False
        self.Comments = self.INN_check+' '+self.Telefon_check+' '+self.Adress_groups_check
        self.debug_check = 0
        self.search_link_url = ''
        #print(self.Telefon_groups_correct[2])
        #print(self.Telefon_groups_correct[3])
        #print(self.Telefon_groups_correct[4])
        #print(self.Telefon_groups_correct[5])
        
    def print_company(self):
        pprint(vars(self))
    
    def convert_to_series(self,row):
        #result['Найдено по'] = 
        #result['Ссылка'] = 
        row[1]['Комментарий'] = (str(self.INN_check) + str(self.Telefon_check) + str(self.Adress_groups_check)).strip()
        return row[1]
    
    def main_search(self,soup):
        """
        Основной поиск по ИНН, телефону и адресу компании на выбранном сайте
        """
        soup = re.sub("[ ]{2,}", " ", soup.get_text())      
        soup = re.sub("[^абвгдежзийклмнопёрстуфхцчшщ\ьъыэюя/0-9 -/]", "", soup,flags=re.I )
        #print(soup)
        if self.INN_check == "":
            try:
                self.INN_result = INN_search(self.INN_patern, soup)
            except:
                self.INN_check = "Критическая ошибка в ИНН"
                self.stop_search = True
                return self
        if self.Telefon_check == '':
            try:
                self.Telefon_groups_result = telefon_search(self.Telefon_patern,soup)
            except:
                self.Telefon_check = 'Критическая ошибка в телефоне'
                self.stop_search = True
                return self
        if self.Adress_groups_check == "":
            try:
                self.Adress_result =  adress_search(self.City,self.Adress_group_1,self.Adress_group_2,soup)
            except:
                self.Adress_groups_check = "Критическая ошибка в адресе"
                self.stop_search = True
                return self
            
        kick_agregator(self,soup)
        
        return self
    
    def check_conditions(self,search_link,row):
        """
        Проверка условий соответствия найденных из поиска данных данным компании
        """
        if self.INN_check == "":
            if self.INN_result is not None:
                row[1]["Найдено по"] = 'Найдено по ИНН'
                row[1]['Ссылка'] = search_link.url
                print(f'По компании {self.Company_name} на сайте {search_link.url} найдено соответсвие')
                self.stop_search = True

        check_telefon_sum = 0
        if self.Telefon_check == '':
            try:
                mis_number = 0
                for first_number in range(0,len(self.Telefon_groups_result)-10):
                    for i,correct_number in enumerate(self.correct_telefon):
                        if correct_number == self.Telefon_groups_result[first_number+i]:   
                            check_telefon_sum +=1
                        elif mis_number == 2:
                            break
                        else:
                            mis_number +=1
                    if check_telefon_sum >= 9:
                        break
                    else:
                        check_telefon_sum = 0
                        mis_number = 0
            except TypeError:
                check_telefon_sum = 0
        if check_telefon_sum >=9:
            row[1]["Найдено по"] = 'Найдено по номеру телефона'
            row[1]['Ссылка'] = search_link.url
            print(f'По компании {self.Company_name} на сайте {search_link.url} найдено соответсвие')
            self.stop_search = True
                                
        if self.Adress_groups_check == "":
            #print(self.Adress_result[0] is not None and self.Adress_result[1] is not None and self.Adress_result[2] is not None)
            if self.Adress_result[0] is not None and self.Adress_result[1] is not None and self.Adress_result[2] is not None:
                #lock.acquire()
                
                row[1]["Найдено по"] = 'Найдено по адресу'
                row[1]['Ссылка'] = search_link.url
                print(f'По компании {self.Company_name} на сайте {search_link.url} найдено соответсвие')
                self.stop_search = True
            
        return self, row
    
    def update_comment(self):
        self.Comments = self.INN_check+' '+ self.Telefon_check+' '+ self.Adress_groups_check
        return self
    def append_additinal_information(self):
        try:
            if self.additional_information != "":
                if self.additional_information.startswith('http://'):
                    self.search_links.insert(0,{'text':'Сайт из выгрузки', 'url': self.additional_information})
                elif self.additional_information.startswith('http://'):
                    self.search_links.insert(0,{'text':'Сайт из выгрузки', 'url': 'http://' + self.additional_information})
                else:
                    self.search_links.insert(0,{'text':'Сайт из выгрузки', 'url': 'http://www.' + self.additional_information})
        except:
            pass
    def debug_me(self):
            if self.debug_check == 1 or self.debug_check == 2:
                print(f'Поиск по компании:  {self.Company_name} \n')
                #print(f'Адрес:  {company.Adress}')
                print('Поиск по сайтам:')
                for search_company in self.search_links:
                    print(search_company['url'])
                print('')
                print('Поиск производится по:')
                print(f'ИНН :{self.INN}')
                print(f'Телефону :{self.correct_telefon}')
                print(f'Город :{self.City}')
                if self.Adress_groups_check != 'Не может перевести адрес в regex':
                    print(f'Первая группа адреса :{self.Adress_group_1}')
                    print(f'Вторая группа адреса :{self.Adress_group_2}\n')
                else: print('Не может перевести адрес в regex\n')

In [6]:
# Вспомогательный класс 

class download_Link():

    def __init__(self,search_link):
        self.search_name = search_link['text']
        self.url = search_link['url']
        self.used_links = set()
        self.cite_links = set()
        self.domain = self.url.split("//")[-1].split("/")[0].split('?')[0]
        self.left_links = set()
        self.search_link_url = ''

    def load_page(self):
        self.used_links.add(self.url) 
        return load_page(self.url)
    

In [7]:
def main(row):
    #try:
        #1) Собираем данные по компании, готовим regex и загружаем сайты из google 
              
        start_time = datetime.datetime.now()
        company = Company_data(row)
        company.debug_check = 0 # 1 если включить дебаг, 2 если нужные еще агрегаторы
        company.debug_me()

        #2) Делаем проход по всем сайтам в списке

        for link in company.search_links:
            
            start_time_search_link = datetime.datetime.now()
            if company.debug_check == 2 or company.debug_check == 1:
                print("Пробует сайт:"+link['url'])
            search_link = download_Link(link)
            soup = search_link.load_page()
            try:
                soup = search_link.load_page()
            except:
                soup = False
                print('По ссылке {} вышла TimeoutError'.format(link['url']))
            #print('вышел из load page')
            
            if soup == False:
                continue
                
            #Проверка на редирект на плохой сайт
            redirect = False
            for avoided_cite in avoided_cites:
                if link['url'].find(avoided_cite) != -1:
                    redirect = True
                    break
            if redirect == True:
                continue
                
            #3) Запускаем поиск на главной странице
            #print('дошел до основного поиска')
            company.search_link_url =search_link.url 
            company = company.main_search(soup)
            #print('вышел из основного поиска')

            #Условие заканчивает алгоритм если критическая ошибка
            if company.stop_search == True:
                row[1]["Найдено по"] = company.update_comment().Comments
                break

            #4) (Если нашел) Выводим по какому полю было найдено значение

            company, row = company.check_conditions(search_link,row)
            if company.stop_search == True:
                break
                
            

            #5) (Если не нашел) Достаем список всех возможных ссылок на сайте  
            

            search_link.cite_links = kick_all_external_links(search_link.domain,soup,search_link.cite_links) 
            search_link.left_links = search_link.cite_links - search_link.used_links
            empty_set = set()

            #print(f'Cite links для компании {company.Company_name}: {search_link.cite_links}')

                #6) Запускаем поиск по сайту
            
            #print('дошел до проверки внутренней части сайта')
            for deep in range(1,level_of_deep+1): 
                if search_link.left_links == empty_set:
                    #print(f'{search_link.domain} Выпал из за empty set: компания {company.Company_name}')
                    break 

                #Продвигаем ссылку с контактами вперед
                search_link.left_links = list(search_link.left_links)
                for left_link in search_link.left_links:
                    if left_link.find('contact') != -1:
                        try:
                            search_link.left_links.remove(left_link)
                            search_link.left_links.insert(0,left_link)
                        except:
                            continue

                for left_link in search_link.left_links:
                    if len(re.findall('/',left_link))-2 == deep:
                        link = left_link
                         
                        if company.debug_check == 2 or company.debug_check == 1:
                            print(f'Используется ссылка {datetime.datetime.now()}: {link}')
                        try:
                            company.search_link_url = link
                            soup = load_page(link)     
                        except:
                            continue
                        finally:
                            search_link.used_links.add(link)
                            search_link.left_links = set(search_link.left_links)
                            search_link.left_links = search_link.cite_links - search_link.used_links

                        #7) запускаем поиск на странице
                        
                        if soup == False:
                            continue

                        company = company.main_search(soup)

                        #8) (Если нашел) Выводим по какому полю было найдено значение 

                        company, row = company.check_conditions(search_link,row)
                        if company.stop_search == True:
                            break
                            
                        if len(search_link.used_links) >= total_number_of_cite_links:
                            #print(f'{search_link.domain} Выпал из за used links: компания {company.Company_name}')
                            break
                            break

                        search_link.cite_links = kick_all_external_links(search_link.domain,soup,search_link.cite_links)
                        search_link.left_links = set(search_link.left_links)
                        search_link.left_links = search_link.cite_links - search_link.used_links 

                    else:
                        continue

                if deep == level_of_deep:
                    #print(f'По компании {company.Company_name} на сайте {search_link.domain} закончилась глубина')
                    break
                if company.stop_search == True:
                    break
 
            end_time_search_link = datetime.datetime.now()
            #print(f'Поиск на сайте {link} занял {end_time_search_link - start_time_search_link} секунд')
            
        if company.stop_search == False:
            row[1]["Найдено по"] = company.Comments
        end_time = datetime.datetime.now()
        row[1]['Время поиска'] = end_time - start_time
        if company.debug_check == 2 or company.debug_check == 1:
            print('')
        print(f'({row[0]}) Конец поиска по компании: {company.Company_name} заняло {end_time - start_time} секунд')
        if company.debug_check == 2 or company.debug_check == 1:
            print('')
        return row[1]
    #except:
     #   row[1]["Найдено по"] = 'Критическая ошибка'
      #  return row[1]

### Загрузка данных из эксель в Python

In [8]:
data = load_data("for alg.xlsx", number_of_rows=100, start_row=0,all_rows=False)
#data = data[data['Название компании'] == 'ШНАЙДЕР ГРУП ХЬЮМАН МЕНЕДЖМЕНТ']
data.reset_index(inplace = True, drop= True)
data

,Название компании,BvDEP ID номер,Номер налогоплательщика (ИНН),Город,Адрес,Номер телефона,Сайт компании,Найдено по,Ссылка,Время поиска,Комментарий
0,Страховой Брокер Малакут,RU51116388,7702417687,Москва,М.Сухаревская пл д.12,+7 495 9331373,malakut.com,Информация не найдена,,,
1,Сигма Полис,RU50929925N,7802822985,Санкт-Петербург,"пр-кт Финляндский д.4, а, офис 404",+7 960 2788111,NaN,Информация не найдена,,,
2,Выбор,RU52205728N,7806499444,Санкт-Петербург,"пр-кт Пискарёвский д.25, лит. р",NaN,NaN,Информация не найдена,,,
3,Обухов-Урал-Консалт,RU85098383,5948034553,д. Хмели,ш. Космонавтов д.332а,+7 342 2403941,NaN,Информация не найдена,,,
4,Эджастерс ПРО,RU26914973N,7724905222,Москва,"проезд Харьковский д. 7, корпус 1, квартира 4",NaN,NaN,Информация не найдена,,,
5,Стаффмаркет,RU40011216N,7730714265,Москва,"ул. Энтузиастов 2-я д. 5, корпус 41, комната 4",NaN,NaN,Информация не найдена,,,
6,Мэйден Раша,RU68973062,7703732057,Москва,"ш. Звенигородское д. 9/27, стр. 1, пом/ком IV/18",NaN,NaN,Информация не найдена,,,
7,Дженсер Центр 2,RU38381944,7728800132,Москва,"пр-кт Новоясеневский д.8, стр.1",+7 495 7862626,NaN,Информация не найдена,,,
8,Партнер-Н,RU81206396,2465112744,Красноярск,ул. 9 Мая д.72,+7 391 2460568,NaN,Информация не найдена,,,
9,Облепиха-Сервис,RU52433560,7713241192,Москва,"ш. Дмитровское д.107, корп.17",+7 495 9951161,NaN,Информация не найдена,,,


### *Блокировка заведомо лишних сайтов*

In [9]:
avoided_cites = """
vk.com
2gis.ru
yandex.ru
list-org.ru
list-org.com
sbis.ru
synapsenet.ru
zachestnyibiznes.ru
spark-interfax.ru
service-online.su
rusprofile.ru
egrinf.com
info.1cont.ru
sotrydnik.com
wikipedia.org
afisha.ru
pro.fira.ru
wikipedia.org
wiktionary.org
dic.academic.ru
wikia.com
yecom.ru
saratov-yel.ru
unfall.ru
teenslang.su
unfall.ru
mvideo.ru
tripadvisor.ru
mosnovostroy.ru
buildlines.ru
kartoteka.ru
basis.myseldon.com
asktel.ru
b2bsky.ru
hh.ru
focus.kontur.ru
k-agent.ru
infobroker.ru
kontragent.skrin.ru
beboss.ru
youtube.com
querycom.ru
khabarovsk7m.ru
yandex.by
audit-it.ru
testfirm.ru
bloomberg
facebook
twitter
vkontakte
instagram
cbonds
egruls.ru
whoiswho.dp
pravda-obman
alladvertising
egrulbox
krasnodar7
synapsenet
egrinf
emis
nalog.io
audit-it
business-rating.company
aliexpress
addresscom
avito
rating.gd
info.1cont
booking.com
indeed.com
zakupki.gov
zoon.ru
amazon
google
tripadvisor
kudago
cian.ru
yandex 
clearspending
/maps/
kinopoisk
org-info
mail.ru
facebook
apsny.today
rushist
адреса.рф
odintsovo
restate
regtorg
azku.ru
tender.pro
yell.ru
napca.ru
cbr.ru
tadviser
http:\www.advoservice.ru
damspravku.ru
"""
avoided_cites = avoided_cites.split('\n')
avoided_cites = [x for x in avoided_cites if x != ""]

### *Блокировка файлов с выбранными расширениям*

In [10]:
avoided_extensions = """
.pdf
.txt
.docx
.docm
.dotx
.doc
.dot
.rtf
.odt
.pdf
.xls
.xlsx
.xlsm
.pptx
.ppt
.exe
.jpg
.rar
"""
avoided_extensions = avoided_extensions.split('\n')
avoided_extensions = [x for x in avoided_extensions if x != ""]

### *Определение глобальных параметров*

In [11]:
level_of_deep = 3 #уровень глубины поиска на сайта
total_number_of_cite_links = 20 #количество страниц на конкретном сайте, которое будет смотреть алгоритм
google_number_of_pages =1 #количество страниц, которые возьмет функция Google
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
tz = pytz.timezone('Europe/Moscow')

### Выполнение алгоритма

### *Мультипроцесс:*

In [12]:
start_time = datetime.datetime.now()
print(f'Начало процесса: {start_time}')
result = []
with Pool() as p:
    result = p.map(main,data.iterrows())
    p.close()
    p.join()
print('Конец мультипроцесса')
end_time = datetime.datetime.now()
print(''.ljust(80,'-'))
print(f'Процесс занял {end_time - start_time} секунд]')

Начало процесса: 2019-08-15 10:03:24.154960
(28) Конец поиска по компании: Мегафинанс Центр Компания заняло 0:00:00.775588 секунд
По компании Страховой Брокер Малакут Созвездие Компания на сайте http://www.malakut.com найдено соответсвие
(30) Конец поиска по компании: Страховой Брокер Малакут Созвездие Компания заняло 0:00:04.585667 секунд
По компании Страховой Брокер Малакут Компания на сайте http://www.malakut.com найдено соответсвие
(0) Конец поиска по компании: Страховой Брокер Малакут Компания заняло 0:00:05.294872 секунд
По компании Страховой Брокер Малакут Созвездие Компания на сайте http://www.malakut.com найдено соответсвие
(29) Конец поиска по компании: Страховой Брокер Малакут Созвездие Компания заняло 0:00:04.552743 секунд
(16) Конец поиска по компании: Премиум-Финанс Компания заняло 0:00:05.975606 секунд
(26) Конец поиска по компании: Капитал-Финанс Компания заняло 0:00:06.159083 секунд
По компании Март Финанс Компания на сайте https://www.sistema-mart.ru/ найдено соответс

In [ ]:
if __name__ == '__main__':
    start_time = datetime.datetime.now()
    print(f'Начало процесса: {start_time}')
    result = []
    with ProcessPool() as p:
        future = p.map(main,data.iterrows(),timeout = 2*google_number_of_pages)
        iterator = future.result()

        p.close()
        p.join()
    print('Конец мультипроцесса')
    end_time = datetime.datetime.now()
    print(''.ljust(80,'-'))
    print(f'Процесс занял {end_time - start_time} секунд]')

### *Один процесс:*

In [27]:
start_time = datetime.datetime.now()
result = list(map(main,data.iterrows()))
print('Конец Обычного процесса')
end_time = datetime.datetime.now()
print(''.ljust(80,'-'))
print(f'Процесс занял {end_time - start_time} секунд]')

(0) Конец поиска по компании: АЦТЕК МЕДИА Компания заняло 0:01:26.970543 секунд
(1) Конец поиска по компании: ЛИБЕРТИ Компания заняло 0:04:02.600384 секунд
Конец Обычного процесса
--------------------------------------------------------------------------------
Процесс занял 0:05:29.573003 секунд]


## Cвод данных в одну таблицу

In [13]:
dataframe = pd.DataFrame(result)
founded = dataframe.index
not_founded = data.drop(founded,axis=0)
not_founded['Найдено по'] = 'Timeerror'
dataframe = dataframe.append(not_founded).sort_index()
#dataframe[dataframe['Ссылка'] != '']
dataframe

,Название компании,BvDEP ID номер,Номер налогоплательщика (ИНН),Город,Адрес,Номер телефона,Сайт компании,Найдено по,Ссылка,Время поиска,Комментарий
0,Страховой Брокер Малакут,RU51116388,7702417687,Москва,М.Сухаревская пл д.12,+7 495 9331373,malakut.com,Найдено по номеру телефона,http://www.malakut.com,00:00:05.294872,
1,Сигма Полис,RU50929925N,7802822985,Санкт-Петербург,"пр-кт Финляндский д.4, а, офис 404",+7 960 2788111,NaN,Был Выбит агрегатор,,00:01:07.239103,
2,Выбор,RU52205728N,7806499444,Санкт-Петербург,"пр-кт Пискарёвский д.25, лит. р",NaN,NaN,Телефон не соотвествует формату Был Выбит агр...,,00:04:57.064235,
3,Обухов-Урал-Консалт,RU85098383,5948034553,д. Хмели,ш. Космонавтов д.332а,+7 342 2403941,NaN,Не может перевести адрес в regexБыл Выбит аг...,,00:01:05.592091,
4,Эджастерс ПРО,RU26914973N,7724905222,Москва,"проезд Харьковский д. 7, корпус 1, квартира 4",NaN,NaN,Телефон не соотвествует формату Был Выбит агр...,,00:32:04.082626,
5,Стаффмаркет,RU40011216N,7730714265,Москва,"ул. Энтузиастов 2-я д. 5, корпус 41, комната 4",NaN,NaN,Телефон не соотвествует формату Был Выбит агр...,,00:05:17.179499,
6,Мэйден Раша,RU68973062,7703732057,Москва,"ш. Звенигородское д. 9/27, стр. 1, пом/ком IV/18",NaN,NaN,Телефон не соотвествует формату Был Выбит агр...,,00:01:17.811464,
7,Дженсер Центр 2,RU38381944,7728800132,Москва,"пр-кт Новоясеневский д.8, стр.1",+7 495 7862626,NaN,Был Выбит агрегаторБыл Выбит агрегаторБыл Вы...,,00:11:23.820736,
8,Партнер-Н,RU81206396,2465112744,Красноярск,ул. 9 Мая д.72,+7 391 2460568,NaN,Не может перевести адрес в regexБыл Выбит аг...,,00:28:21.117721,
9,Облепиха-Сервис,RU52433560,7713241192,Москва,"ш. Дмитровское д.107, корп.17",+7 495 9951161,NaN,Был Выбит агрегаторБыл Выбит агрегатор,,00:03:31.796452,


In [ ]:
result

## Выгрузка данных в excel

In [18]:
dataframe.to_excel('бенч маркетинг.xlsx')

## Технические проверки